**Vous allez effectuer une classification supervisée sur la colonne "survived", en utilisant l'arbre de décision du classifieur. Vous devez faire apparaître clairement le score d'exactitude du jeu de test .**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Charger les données Titanic
url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/titanic.csv"
df = pd.read_csv(url)

# Supprimer les colonnes inutiles
df = df.drop(columns=['Name'])

# Remplacer les valeurs manquantes
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)

# Convertir les variables catégoriques en numériques
df = pd.get_dummies(df, columns=['Sex'], drop_first=True)


X = df.drop(columns=['Survived'])
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardisation des données
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entraînement du modèle
clf = DecisionTreeClassifier()
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"🔹 Score d'accuracy sur le test : {accuracy:.2f}")

🔹 Score d'accuracy sur le test : 0.72


<ipython-input-2-bc34adbdd5ca>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-2-bc34adbdd5ca>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

**Avec ce même classificateur, vous allez lancer un GridSearch avec les valeurs :
tous les entiers de 1 à 50 du paramètremax_depth
tous les entiers entre 1 et 15 du paramètremin_samples_leaf
(2, 5, 7, 10, 15, 30) du paramètremin_samples_split**

In [7]:
from sklearn.model_selection import GridSearchCV

# Définition de la grille d'hyperparamètres
param_grid = {
    'max_depth': list(range(1, 51)),
    'min_samples_leaf': list(range(1, 16)),
    'min_samples_split': [2, 5, 7, 10, 15, 30]
}


clf = DecisionTreeClassifier()
grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(X_train_scaled, y_train)

best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("🔹 Meilleurs hyperparamètres avec GridSearch :")
print(best_params)
print(f"🔹 Meilleur score (validation croisée) : {best_score:.2f}")


🔹 Meilleurs hyperparamètres avec GridSearch :
{'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 5}
🔹 Meilleur score (validation croisée) : 0.83


**Avec ce même classifieur, vous allez lancer un RandomSearch sur les mêmes valeurs des paramètres en limitant à 200 itérations.**

In [6]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


param_dist = {
    'max_depth': list(range(1, 51)),
    'min_samples_leaf': list(range(1, 16)),
    'min_samples_split': [2, 5, 7, 10, 15, 30]
}

# RandomizedSearch avec 200 itérations
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=200)

random_search.fit(X_train_scaled, y_train)

# Meilleurs paramètres trouvés
best_params_random = random_search.best_params_
best_score_random = random_search.best_score_

print("🔹 Meilleurs hyperparamètres avec RandomizedSearch :")
print(best_params_random)
print(f"🔹 Meilleur score (validation croisée) : {best_score_random:.2f}")


🔹 Meilleurs hyperparamètres avec RandomizedSearch :
{'min_samples_split': 7, 'min_samples_leaf': 8, 'max_depth': 4}
🔹 Meilleur score (validation croisée) : 0.83
